In [ ]:
import lux
lux.config.lazy_maintain = True
lux.config.early_pruning = True
lux.config.streaming = True

This notebook is originally derived from [this notebook on Kaggle](https://www.kaggle.com/kkanda/analyzing-uci-crime-and-communities-dfset).

In [ ]:
# {{NO LUX}}
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split,KFold,cross_val_score
import re
import json
from pandas.io.json import json_normalize

In [ ]:
numPoints=100

In [ ]:
# {{NO LUX}}
df = pd.read_csv("../../data/communities_100k.csv",na_values=["?"]) 
df = df.sample(n=int(numPoints),random_state=111)

In [ ]:
# {{NO LUX}}
df.columns

In [ ]:
# {{PRINT SERIES}}
df.ViolentCrimesPerPo

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{NO LUX}}
df = df.rename(columns={'Êcommunityname':'communityName'})

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{PRINT SERIES}}
df.communityname

In [ ]:
# {{NO LUX}}
print (list(df.columns))

In [ ]:
# {{NO LUX}}
df.info()

In [ ]:
# {{NO LUX}}
cols = ['HousVacant','PctHousOccup','PctHousOwnOcc','PctVacantBoarded','PctVacMore6Mos','PctUnemployed','PctEmploy','ViolentCrimesPerPo']
df.fillna(df[cols].median(),inplace=True)

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{NO LUX}}
df_state = df.groupby('state').agg({'ViolentCrimesPerPo':'mean'})[['ViolentCrimesPerPo']].reset_index()

In [ ]:
# {{PRINT DF}}
df_state

In [ ]:
# {{NO LUX}}
cols = ['HousVacant','PctHousOccup','PctHousOwnOcc','PctVacantBoarded','PctVacMore6Mos','PctUnemployed','PctEmploy','murdPerPop','rapesPerPop','robbbPerPop','assaultPerPop','ViolentCrimesPerPo','burglPerPop','larcPerPop','autoTheftPerPop','arsonsPerPop','nonViolPerPop']
df_study = df.filter(cols, axis=1)

In [ ]:
# {{PRINT DF}}
df_study

In [ ]:
# {{NO LUX}}
corr_df_study = df_study.corr()

In [ ]:
# {{PRINT DF}}
corr_df_study

The latter half of this notebook is originally derived from [this notebook](https://github.com/vbordalo/Communities-Crime/blob/master/Crime_v1.ipynb).

In [ ]:
# {{NO LUX}}
df = df.drop(columns=['state','county',
                          'community','communityname',
                          'fold'], axis=1)

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{NO LUX}}
df = df.replace('?', np.nan)
feat_miss = df.columns[df.isnull().any()]

In [ ]:
# {{NO LUX}}
print(feat_miss)
feat_miss.shape

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{PRINT DF}}
df[feat_miss[13:23]].describe()

In [ ]:
# {{NO LUX}}
df = df.dropna(axis=1)

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{PRINT DF}}
df.describe()

In [ ]:
# {{NO LUX}}
df.hist(column = ['ViolentCrimesPerPo'], bins = 30, color = 'red', alpha = 0.8)
plt.show()

In [ ]:
# {{NO LUX}}
import seaborn as sns

corrmat = df.corr()
fig = plt.figure(figsize = (16, 12))

sns.heatmap(corrmat, vmax = 0.8)
plt.show()

In [ ]:
# {{NO LUX}}
corrT = df.corr(method = 'pearson').round(4)

In [ ]:
# {{PRINT DF}}
corrT

In [ ]:
# {{NO LUX}}
corrT = corrT.sort_values(by=['ViolentCrimesPerPo'])

In [ ]:
# {{PRINT DF}}
corrT

In [ ]:
# {{PRINT SERIES}}
corrT['ViolentCrimesPerPo']

In [ ]:
# {{NO LUX}}
X = df.iloc[:, 0:100].values
y = df["ViolentCrimesPerPo"]

In [ ]:
# {{PRINT SERIES}}
y

In [ ]:
# {{NO LUX}}
from sklearn.model_selection import train_test_split
seed = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
X_train

In [ ]:
# {{NO LUX}}
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# {{PRINT DF}}
df

In [ ]:
# {{NO LUX}}
from sklearn.decomposition import PCA

c = 14
pca = PCA(n_components = c)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print("Amount of variance: %s" % pca.explained_variance_)
print("Sum of the variance: %s" % sum(pca.explained_variance_).round(2))

print("Percentage of variance: %s" % pca.explained_variance_ratio_)
print("Sum of the percentage of variance: %s" % sum(pca.explained_variance_ratio_).round(2))


plt.scatter(np.arange(1,(c+1)),pca.explained_variance_, c = 'red')
plt.plot((0,15), (1,1), color = 'black', linestyle = 'dashed')
plt.xlabel('PC')
plt.ylabel('Amount of variance explained')
plt.show()
print(X_train.shape)

In [ ]:
# {{NO LUX}}
def myplot(XX, yy):
    from sklearn.model_selection import ShuffleSplit
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import make_scorer
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=seed)

    results = []
    names = []

    for name, model in models:
        cv_results = cross_val_score(model, XX, yy, cv = cv, scoring = make_scorer(r2_score))
        results.append(cv_results)
        names.append(name)
        msg = "%s: %.3f (+/- %.3f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

    fig = plt.figure()
    fig.suptitle('R2')
    ax = fig.add_subplot(111)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

In [ ]:
# {{NO LUX}}
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

models = []
models.append(('LR', LinearRegression()))
models.append(('SVR', SVR()))
models.append(('DTR', DecisionTreeRegressor(random_state=seed)))
models.append(('RFR (100 Trees)', RandomForestRegressor(n_estimators=100, random_state=seed)))

In [ ]:
# {{NO LUX}}
myplot(X_train, y_train)

In [ ]:
# {{NO LUX}}
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor

#model = LinearRegression(fit_intercept=True)
model = RANSACRegressor(LinearRegression(),
                         max_trials=100,
                         min_samples=50,
                         loss='absolute_loss',
                         residual_threshold=0.55,
                         random_state=seed)


model.fit(X_train, y_train)

y_pred = model.predict(X_train)
# Correction: y_pred (per capita violent crimes) can not assume negative values
y_pred[y_pred < 0] = 0

K = X_train.shape[1]+1
rms = np.sqrt(sum((y_train-y_pred)**2)/(len(y_train)-K))

In [ ]:
# {{NO LUX}}
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('R^2: %.4f' % r2_score(y_train,y_pred))
print('MSE: %.4f' % mean_squared_error(y_train,y_pred))
print('MAE: %.4f' % mean_absolute_error(y_train,y_pred))
print('rms: %.4f' % rms)

inlier_mask = model.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

plt.figure(figsize=(15, 10))
ax1 = plt.subplot(121)
ax1.scatter(y_pred[inlier_mask], y_train[inlier_mask], c = 'blue', alpha = 0.5)
ax1.scatter(y_pred[outlier_mask], y_train[outlier_mask], c = 'red', alpha = 0.5)
ax1.plot((-0.1,1.2),(-0.1,1.2), c = 'black', linestyle = 'dashed')
ax1.set_title('Training Set')
ax1.set_xlabel('y_pred_train')
ax1.set_ylabel('y_train')

ax2 = plt.subplot(122)
ax2.scatter(y_pred[inlier_mask], y_pred[inlier_mask] - y_train[inlier_mask], c = 'blue', alpha = 0.5)
ax2.scatter(y_pred[outlier_mask], y_pred[outlier_mask] - y_train[outlier_mask], c = 'red', alpha = 0.5)
ax2.plot((0,1.2),(0,0), c = 'black', linestyle = 'dashed')
ax2.set_title('Training Set')
ax2.set_xlabel('y_pred_train')
ax2.set_ylabel('Residuals')

plt.subplots_adjust(wspace=0.4, top=0.5)
plt.show()

In [ ]:
# {{NO LUX}}
y_pred_test = model.predict(X_test)
y_pred_test[y_pred_test < 0] = 0

nrp = X_test.shape[1]+1
rms = np.sqrt(sum((y_test-y_pred_test)**2)/(len(y_test)-nrp))

print('R^2: %.4f' % r2_score(y_test,y_pred_test))
print('MSE: %.4f' % mean_squared_error(y_test,y_pred_test))
print('MAE: %.4f' % mean_absolute_error(y_test,y_pred_test))
print('rms: %.4f' % rms)

plt.figure(figsize=(15, 10))
ax1 = plt.subplot(121)
ax1.scatter(y_pred_test, y_test, c = 'green', alpha = 0.5)
ax1.plot((-0.1,1.2),(-0.1,1.2), c = 'black', linestyle = 'dashed')
ax1.set_title('Test Set')
ax1.set_xlabel('y_pred_test')
ax1.set_ylabel('y_test')

ax2 = plt.subplot(122)
ax2.scatter(y_pred_test, y_pred_test - y_test, c = 'green', alpha = 0.5)
ax2.plot((0,1.2),(0,0), c = 'black', linestyle = 'dashed')
ax2.set_title('Test Set')
ax2.set_xlabel('y_pred_test')
ax2.set_ylabel('Residuals')

plt.subplots_adjust(wspace=0.4, top=0.5)
plt.show()

In [ ]:
# {{NO LUX}}
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor

model = RANSACRegressor(LinearRegression(),
                         max_trials=100,
                         min_samples=50,
                         loss='absolute_loss',
                         residual_threshold=0.55,
                         random_state=seed)


model.fit(X_train, y_train)

y_pred = model.predict(X_train)
# Correction: y_pred (per capita violent crimes) can not assume negative values
y_pred[y_pred < 0] = 0

K = X_train.shape[1]+1
rms = np.sqrt(sum((y_train-y_pred)**2)/(len(y_train)-K))

In [ ]:
# {{NO LUX}}
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('R^2: %.4f' % r2_score(y_train,y_pred))
print('MSE: %.4f' % mean_squared_error(y_train,y_pred))
print('MAE: %.4f' % mean_absolute_error(y_train,y_pred))
print('rms: %.4f' % rms)

inlier_mask = model.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

plt.figure(figsize=(15, 10))
ax1 = plt.subplot(121)
ax1.scatter(y_pred[inlier_mask], y_train[inlier_mask], c = 'blue', alpha = 0.5)
ax1.scatter(y_pred[outlier_mask], y_train[outlier_mask], c = 'red', alpha = 0.5)
ax1.plot((-0.1,1.2),(-0.1,1.2), c = 'black', linestyle = 'dashed')
ax1.set_title('Training Set')
ax1.set_xlabel('y_pred_train')
ax1.set_ylabel('y_train')

ax2 = plt.subplot(122)
ax2.scatter(y_pred[inlier_mask], y_pred[inlier_mask] - y_train[inlier_mask], c = 'blue', alpha = 0.5)
ax2.scatter(y_pred[outlier_mask], y_pred[outlier_mask] - y_train[outlier_mask], c = 'red', alpha = 0.5)
ax2.plot((0,1.2),(0,0), c = 'black', linestyle = 'dashed')
ax2.set_title('Training Set')
ax2.set_xlabel('y_pred_train')
ax2.set_ylabel('Residuals')

plt.subplots_adjust(wspace=0.4, top=0.5)
plt.show()